# Creating an InterMine workflow using the API

We are going to re-create the workflow we did using the web interface using the python API.

We start by importing the Service class from InterMine's webservice module.  You will need to access your account on humanMine and you do this through an API token. You can get your token by logging into [HumanMine](http://www.humanmine.org/) and going to the account details tab within MyMine.  Cut and paste your token into the code below.

In [1]:
from intermine.webservice import Service
service = Service("https://www.humanmine.org/humanmine/service", token = "YOUR TOKEN HERE")


ServiceError: "Could not validate service - is the root url (https://www.humanmine.org/humanmine/service) correct? [Errno Insufficient permissions - Token YOUR TOKEN HERE] 401: ''"

Our first query looked at whether the set of Pax6 targets (from list PL_Pax6_Targets) are expressed in the pancreas.  In the web interface we used a template to run this query, but here we will create a query object. 

In [ ]:
query = service.new_query("Gene")


First we will define the output columns that we want in our result - i.e the view. We want to add fields (attributes) from both the Gene class and the proteinAtlasExpression class.

In [ ]:
query.add_view(
    "primaryIdentifier", "symbol", "proteinAtlasExpression.cellType",
    "proteinAtlasExpression.level", "proteinAtlasExpression.reliability",
    "proteinAtlasExpression.tissue.name"
)



Next, add the constraints to your query. We want to constrain the Gene class to the genes in the PL_Pax6_Targets list.

In [ ]:
query.add_constraint("Gene", "IN", "PL_Pax6_Targets", code = "A")


We also need to constrain the expression level to be "high" or "medium" and the tissue to be "pancreas".

In [ ]:
query.add_constraint("proteinAtlasExpression.tissue.name", "=", "Pancreas", code = "B")
query.add_constraint("proteinAtlasExpression.level", "ONE OF", ["Medium", "High"], code = "C")


Now, let's check what the query returns by looping through the rows and printing the results:

In [ ]:
for row in query.rows():
    print(row["primaryIdentifier"], row["symbol"], row["proteinAtlasExpression.cellType"], \
        row["proteinAtlasExpression.level"], row["proteinAtlasExpression.reliability"], \
        row["proteinAtlasExpression.tissue.name"])

We want to save this set of genes (i.e genes from the Pax6 target set that are expressed in the pancreas) for further analysis. To do this we define our python list and loop through our results again - this time, instead of printing the results, we append just the primary identifiers returned to our list.

In [ ]:
UpinPancreas = list()
for row in query.rows():
    UpinPancreas.append(row["primaryIdentifier"])

and check that the list we have created looks correct:

In [ ]:
print(UpinPancreas)

We now need to save the list to our intermine account so we can use it again in a later query. The ListManager class provides methods to manage list contents and operations.

In [ ]:
lm=service.list_manager()
lm.create_list(content=UpinPancreas, list_type="Gene", name="UpinPancreas")

Log in to HumanMine and check your list has been created.

## Second query: Diabetes genes 

Our second query (which we created using the query builder) found genes that are associated with the diesease diabetes.  Re-create this query using code as follows:

In [ ]:
query2 = service.new_query("Gene")
query2.add_view("primaryIdentifier", "symbol")
query2.add_constraint("organism.name", "=", "Homo sapiens", code = "A")
query2.add_constraint("diseases.name", "CONTAINS", "diabetes", code = "B")

for row in query2.rows():
    print (row["primaryIdentifier"], row["symbol"])

and save the set of genes returned as a list:

In [ ]:
diabetesGenes = list()
for row in query2.rows():
    diabetesGenes.append(row["primaryIdentifier"])

In [ ]:
lm=service.list_manager()
lm.create_list(content=diabetesGenes, list_type="Gene", name="diabetesGenes")

Next, we used a list intersect to find those genes that are upregulated in the pancreas that are also associated with the disease diabetes. We need to intersect the first (UpinPancreas) and second (diabetesGenes) lists that we created. We can do this using the intersect method from the ListManager class.

In [ ]:
lm.intersect(["UpinPancreas", "diabetesGenes"], "intersectedList")

In [ ]:
intersectedList = lm.get_list("intersectedList")

In [ ]:
print(intersectedList)

## Final Query: GWAS 

Finally, we fed the intersected list from above back into another query to see if there was any association of these genes with diabetes phenotypes according to GWAS studies. Note that we now start our query from the GWAS class:

In [ ]:
query = service.new_query("GWAS")

In [ ]:
query.add_view(
    "results.associatedGenes.primaryIdentifier",
    "results.associatedGenes.symbol", "results.associatedGenes.name",
    "results.SNP.primaryIdentifier", "results.pValue", "results.phenotype",
    "firstAuthor", "name", "publication.pubMedId",
    "results.associatedGenes.organism.shortName"
)


In [ ]:
query.add_constraint("results.pValue", "<=", "1e-04", code = "B")
query.add_constraint("results.phenotype", "CONTAINS", "diabetes", code = "C")
query.add_constraint("results.associatedGenes", "IN", "intersectedList", code = "D")

In [ ]:
for row in query.rows():
    print(row["results.associatedGenes.primaryIdentifier"], row["results.associatedGenes.symbol"], \
        row["results.associatedGenes.name"], row["results.SNP.primaryIdentifier"], \
        row["results.pValue"], row["results.phenotype"], row["firstAuthor"], row["name"], \
        row["publication.pubMedId"], row["results.associatedGenes.organism.shortName"])

In [ ]:
for row in query.rows():
    print(row["results.associatedGenes.symbol"])